In [2]:
### Importing the libaries for image processing

import cv2
import numpy
import tensorflow
from PIL import Image
import os

2025-12-09 23:37:55.177781: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-09 23:37:55.189837: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-09 23:37:55.607617: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-09 23:37:57.193668: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [ ]:
class ImageHandler:
    
    def validate_format(self, path):
        valid_exts = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
        _, ext = os.path.splitext(path)
        if ext.lower() not in valid_exts:
            raise ValueError(f"Invalid file extension: {ext}")

    def load_image(self, path):
        # 1. Check if file exists
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")

        # 2. Validate format
        self.validate_format(path)

        # 3. Load image
        img = cv2.imread(path)

        # 4. specific check: OpenCV returns None if it fails to decode
        if img is None:
            raise ValueError("Could not decode image. File might be corrupted.")

        return img

    def save_image(self, img, path):
        success = cv2.imwrite(path, img)
        if not success:
            raise IOError(f"Failed to save image to {path}")

    def resize_image(self, img, max_dim=1280):
        h, w = img.shape[:2]
        
        # Only resize if the image is actually bigger than max_dim
        if max(h, w) > max_dim:
            scale = max_dim / max(h, w)
            new_w = int(w * scale)
            new_h = int(h * scale)
            return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
            
        return img

    def to_rgb(self, img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def preprocess_for_model(self, path):
        """Loads, Resizes, and Converts to RGB in one go."""
        img = self.load_image(path)
        img = self.resize_image(img, max_dim=224) # Standard ML size
        img = self.to_rgb(img)
        return img

In [16]:
img = ImageHandler()

import sys
import os

# Automatically get the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(project_root)

print(project_root + "/data/raw/image.png")

img.load_image("image.png")


/home/wayne/Github/PrivacyEnc/data/raw/image.png
Import Successful
